In [1]:
import numpy as np
import random
import pickle
import time

In [3]:
learning_rate=0.01 #alpha
decay_gamma=0.7 #gamma
exp_rate=0.3 #epsilon
starting_player='agent'
list_of_winners=[]
policy={}
initial_board=[3,5]

In [4]:
class NimGame:
    def __init__(self, init_board):
        self.board=initial_board.copy()
        self.number_of_piles=np.count_nonzero(self.board)
        self.player=starting_player
        self.winner=None
        self.starting_player=starting_player
        self.states_value={}
        self.states=[]
        self.exp_rate=exp_rate
        self.lr=learning_rate
        self.gamma=decay_gamma
    def change_player(self):
        """A function for changing player"""
        if self.player=='agent':
            self.player='dummy'
        else:
            self.player='agent'
    def get_reward(self):
        """This function determines the reward to give.
            Returns the reward."""
        if self.player =='agent':
            if sum(self.board)==1:
                return 1 
        if self.is_game_over():
            if self.player == 'agent': #if player 1 takes last stick
                return -1
            else:
                return 1 # in player 2 takes last stick
        else:
            return 0 #if game is not over
    def get_legal_actions(self):
        """This function determines all legal actions on the current board state.
            Returns a list of legal actions."""
        actions = []
        for i in np.nonzero(self.board)[0]:#indices of non-zero piles
            for j in range(1, self.board[i]+1): #number of stones
                actions.append((i, j))
        return actions
    def is_game_over(self):
        """A function for determining whether the game is over.
            Returns True if the game is over and False otherwise"""
        return sum(self.board)==0
    def possible_next_states(self, state):
        """Function to consider all possible board states which is obtainable from a given state.
            Input is a board state for which you want to find all possible obtainable states from.
            Returns a list of all possible obtainable board states."""
        list_to_return=[]
        for i in range(state[0]+1):
            for j in range(state[1]+1):
                list_to_return.append([state[0]-i, state[1]-j])
        if state in list_to_return:
            list_to_return.remove(state)
        return list_to_return
    def feedReward(self, reward):
        """Function for updating the Q-table"""
        for st in reversed(self.states):  # goes through all saved board states of this game
            possible_next_st=self.possible_next_states(st)
            temp=-1000
            for i in possible_next_st:
                if self.states_value.get(str(i)) is not None and self.states_value.get(str(i))>temp:
                    temp=self.states_value.get(str(i))
            if self.states_value.get(str(st)) is None: 
                self.states_value[str(st)] = 0  # initialise a value for the state
            self.states_value[str(st)] += self.lr * (reward -self.gamma*temp- self.states_value[str(st)])
    def reset(self,init_state):
        """A function for resetting the game after an ended game."""
        self.board=init_state
        self.states=[]
        self.number_of_piles=np.count_nonzero(self.board)
        self.player=self.starting_player
        self.winner=None
        return
    def nim_sum(self, numbers):
        """Function for calculating the nim sum.
            Input is a list of numbers.
            Returns Nim sum of the numbers in the list."""
        binary_numbers = [bin(num)[2:].zfill(len(bin(max(numbers))) - 2) for num in numbers]
        column_sums = [sum(int(binary[i]) for binary in binary_numbers) for i in range(len(binary_numbers[0]))]
        nim_sum = ''.join(['0' if sum % 2 == 0 else '1' for sum in column_sums])
        return int(nim_sum, 2)
    def choose_optimal_action(self):
        """This function returns the optimal action, i.e. such that the nim sum is zero.
        if this is not possible, then it returns a random action"""
        if nim_sum(self.board)==0:
            return self.choose_random_action()
        else:
            actions=self.get_legal_actions()
            for a in actions:
                test_board=self.board.copy()
                pile_number, amount_to_remove=a
                test_board[pile_number]-=amount_to_remove
                if nim_sum(test_board)==0:
                    return a
    def choose_random_action(self):
        """A function for choosing an action randomly.
            Returns an action."""
        action=self.get_legal_actions()
        return random.choice(action)
    def choose_smart_action(self):
        """A function for choosing an action according to the Q-table.
        Returns an action."""
        actions=self.get_legal_actions()
        value_max = -999
        for p in actions:
            board_copy=self.board.copy()
            pile_number, amount_to_remove=p
            board_copy[pile_number]-=amount_to_remove
            next_board = board_copy
            value = 0 if self.states_value.get(str(next_board)) is None else self.states_value.get(str(next_board))
            if value >= value_max:
                value_max = value
                move = p
        return move
    def play_action(self, action):
        """play a given action."""
        pile_number, amount_to_remove=action
        self.board[pile_number]-=amount_to_remove
        return
    def training_game(self, rounds=10):
        """The function for training the agent.
            Input is the number of training games to be played"""
        for i in range(rounds):
            while not self.is_game_over():
                if self.player=='dummy':
                    move= self.choose_random_action()
                else:
                    if np.random.uniform(0,1)<= self.exp_rate:
                        move = self.choose_random_action()
                    else:
                        move=self.choose_smart_action()
                self.play_action(move)
                if self.player=='agent':
                    self.states.append(self.board.copy())
                if self.is_game_over():
                    self.winner=self.player
                    list_of_winners.append(self.winner)
                    if self.winner=='agent':
                        self.feedReward(5)
                    else:
                        self.feedReward(-1)
                    self.reset(initial_board.copy())
                    break
                else:
                    self.change_player()
    def train_against_pro(self, rounds=1000):
        """In this function, the agent is trained against a player who only performs optimal moves
        - note that this player is still named dummy.
            Input is the number of training games to be played."""
        for i in range(rounds):
            while not self.is_game_over():
                if self.player=='dummy':
                    move= self.choose_optimal_action()
                else:
                    if np.random.uniform(0,1)<= self.exp_rate:
                        move = self.choose_random_action()
                    else:
                        move=self.choose_smart_action()
                self.play_action(move)
                if self.player=='agent':
                    self.states.append(self.board.copy())
                    #print(self.board, self.states, 'hej')
                if self.is_game_over():
                    self.winner=self.player
                    list_of_winners.append(self.winner)
                    if self.winner=='agent':
                        self.feedReward(100)
                    else:
                        self.feedReward(-1)
                    self.reset(initial_board.copy())
                    break
                else:
                    self.change_player()
    def test_against_pro(self, rounds=1000):
        """A function for testing a given policy against a player performing optimal moves.
            Input is the number of test games to be played."""
        for i in range(rounds):
            while not self.is_game_over():
                if self.player=='dummy':
                    move= self.choose_optimal_action()
                    #print('dummy chooses the following move', move)
                else:
                        move=self.choose_smart_action()
                        #print('agent chooses the following move', move)
                self.play_action(move)
                #print('the board is now:', self.board, 'with nim sum:', self.nim_sum(self.board))
                if self.is_game_over():
                    self.winner=self.player
                    list_of_winners.append(self.winner)
                    self.reset(initial_board.copy())
                    break
                else:
                    self.change_player()
    def test_game(self, rounds=1000):
        """A function for testing a policy against a player with the random strategy.
            Input is the number of test game to be played."""
        for i in range(rounds):
            while not self.is_game_over():
                if self.player=='dummy':
                    move= self.choose_random_action()
                else:
                        move=self.choose_smart_action()
                self.play_action(move)
                if self.is_game_over():
                    self.winner=self.player
                    list_of_winners.append(self.winner)
                    self.reset(initial_board.copy())
                    break
                else:
                    self.change_player()


In [7]:
list_of_alphas=[0.01*i for i in range(1,20,2)]
list_of_gammas=[0.1*i for i in range(1,10,2)]
list_of_epsilons=[0.1*i for i in range(1,10,2)]
list_of_epsilons

[0.1, 0.30000000000000004, 0.5, 0.7000000000000001, 0.9]

In [ ]:
list_of_alphas=[0.01*i for i in range(1,20,2)]
list_of_gammas=[0.1*i for i in range(1,10,2)]
list_of_epsilons=[0.1*i for i in range(1,10,2)]
for alpha in list_of_alphas:
    for gamma in list_of_gammas:
        for epsilon in list_of_epsilons:
            initial_board=[1,3,5]
            nim=NimGame(initial_board)
            nim.lr=alpha
            nim.exp_rate=epsilon
            nim.gamma=gamma
            number_of_iterations=10
            rounds_mean=[]
            time_mean=[]
            for i in range(number_of_iterations):
                list_of_winners=[]
                policy={}
                nim.states_value=policy
                total_rounds=0
                rounds=1000
                start_time=time.time()
                while list_of_winners.count('agent')*100/rounds<100:
                    list_of_winners=[]
                    nim.train_against_pro(rounds)
                    list_of_winners=[]
                    nim.test_against_pro(rounds)
                    total_rounds+=rounds
                    #print('agent wins:', list_of_winners.count('agent')*100/rounds, total_rounds)
                    if list_of_winners.count('agent')*100/rounds>max_win_rate:
                        max_win_rate=list_of_winners.count('agent')*100/rounds
                end_time=time.time()
                rounds_mean.append(total_rounds)
                time_mean.append(end_time-start_time)
                #print(total_rounds, end_time-start_time)
            
            #print('In average, it took', np.mean(rounds_mean),'training games and', np.mean(time_mean), 'seconds to find an optimal policy.')
            #print('alpha:',alpha, ', gamma:', gamma, ', epsilon:', epsilon)
            print(np.mean(rounds_mean), np.mean(time_mean), alpha, gamma, epsilon)

### This is what the program printed


54400.0 6.605865263938904 0.01 0.1 0.1
20400.0 2.546582651138306 0.01 0.1 0.30000000000000004
16800.0 2.218331456184387 0.01 0.1 0.5
16100.0 2.0976560592651365 0.01 0.1 0.7000000000000001
26900.0 3.4539296865463256 0.01 0.1 0.9
91400.0 11.223314332962037 0.01 0.30000000000000004 0.1
38200.0 4.798337125778199 0.01 0.30000000000000004 0.30000000000000004
30400.0 3.9691399335861206 0.01 0.30000000000000004 0.5
33600.0 4.340031218528748 0.01 0.30000000000000004 0.7000000000000001
56600.0 7.267593359947204 0.01 0.30000000000000004 0.9
115000.0 14.055317282676697 0.01 0.5 0.1
47800.0 5.960411119461059 0.01 0.5 0.30000000000000004
37900.0 4.796879696846008 0.01 0.5 0.5
41400.0 5.345577955245972 0.01 0.5 0.7000000000000001
75500.0 9.705233931541443 0.01 0.5 0.9
132400.0 16.89210138320923 0.01 0.7000000000000001 0.1
52500.0 6.695010948181152 0.01 0.7000000000000001 0.30000000000000004
41600.0 5.380018997192383 0.01 0.7000000000000001 0.5
46600.0 6.018443536758423 0.01 0.7000000000000001 0.7000000000000001
83900.0 10.811143279075623 0.01 0.7000000000000001 0.9
137200.0 16.910699820518495 0.01 0.9 0.1
56800.0 7.34582633972168 0.01 0.9 0.30000000000000004
46800.0 6.136266899108887 0.01 0.9 0.5
51400.0 6.633748483657837 0.01 0.9 0.7000000000000001
95300.0 12.253316402435303 0.01 0.9 0.9
17700.0 2.1876007556915282 0.03 0.1 0.1
7600.0 0.9500716209411622 0.03 0.1 0.30000000000000004
5900.0 0.757445502281189 0.03 0.1 0.5
6100.0 0.7888650178909302 0.03 0.1 0.7000000000000001
10400.0 1.3304452657699586 0.03 0.1 0.9
32900.0 4.062986660003662 0.03 0.30000000000000004 0.1
13400.0 1.700400161743164 0.03 0.30000000000000004 0.30000000000000004
10500.0 1.349242877960205 0.03 0.30000000000000004 0.5
11200.0 1.4565393924713135 0.03 0.30000000000000004 0.7000000000000001
20900.0 2.6951905727386474 0.03 0.30000000000000004 0.9
36900.0 4.610458731651306 0.03 0.5 0.1
16500.0 2.0809727907180786 0.03 0.5 0.30000000000000004
12100.0 1.5664000988006592 0.03 0.5 0.5
14700.0 1.924274754524231 0.03 0.5 0.7000000000000001
23300.0 3.014676308631897 0.03 0.5 0.9
41200.0 5.463363742828369 0.03 0.7000000000000001 0.1
17900.0 2.282658004760742 0.03 0.7000000000000001 0.30000000000000004
14300.0 1.8537375211715699 0.03 0.7000000000000001 0.5
15300.0 1.9965930461883545 0.03 0.7000000000000001 0.7000000000000001
25700.0 3.332640218734741 0.03 0.7000000000000001 0.9
45000.0 5.5858386039733885 0.03 0.9 0.1
20000.0 2.5564598560333254 0.03 0.9 0.30000000000000004
14700.0 1.9006131172180176 0.03 0.9 0.5
17700.0 2.288687562942505 0.03 0.9 0.7000000000000001
31200.0 4.034735250473022 0.03 0.9 0.9
14000.0 1.7115816354751587 0.05 0.1 0.1
4900.0 0.6270369768142701 0.05 0.1 0.30000000000000004
3500.0 0.4442156791687012 0.05 0.1 0.5
3200.0 0.409309983253479 0.05 0.1 0.7000000000000001
6100.0 0.7857693433761597 0.05 0.1 0.9
19400.0 2.387915086746216 0.05 0.30000000000000004 0.1
8300.0 1.070654296875 0.05 0.30000000000000004 0.30000000000000004
6600.0 0.8497924089431763 0.05 0.30000000000000004 0.5
6700.0 0.8678466796875 0.05 0.30000000000000004 0.7000000000000001
11400.0 1.4645888090133667 0.05 0.30000000000000004 0.9
23100.0 2.895168995857239 0.05 0.5 0.1
9900.0 1.2450314521789552 0.05 0.5 0.30000000000000004
7800.0 1.0132437229156495 0.05 0.5 0.5
8700.0 1.116224217414856 0.05 0.5 0.7000000000000001
14300.0 1.8363401174545289 0.05 0.5 0.9
25900.0 3.230182147026062 0.05 0.7000000000000001 0.1
11600.0 1.460542368888855 0.05 0.7000000000000001 0.30000000000000004
8600.0 1.1193601369857789 0.05 0.7000000000000001 0.5
9600.0 1.2438376665115356 0.05 0.7000000000000001 0.7000000000000001
16800.0 2.156690526008606 0.05 0.7000000000000001 0.9
27300.0 3.3777499437332152 0.05 0.9 0.1
12400.0 1.5758662939071655 0.05 0.9 0.30000000000000004
9500.0 1.215079426765442 0.05 0.9 0.5
10500.0 1.3591228246688842 0.05 0.9 0.7000000000000001
20300.0 2.606812524795532 0.05 0.9 0.9
9400.0 1.161474919319153 0.07 0.1 0.1
3700.0 0.47504923343658445 0.07 0.1 0.30000000000000004
3000.0 0.3767707109451294 0.07 0.1 0.5
3100.0 0.39662353992462157 0.07 0.1 0.7000000000000001
4800.0 0.6160403966903687 0.07 0.1 0.9
13000.0 1.6120084285736085 0.07 0.30000000000000004 0.1
6100.0 0.7883914947509766 0.07 0.30000000000000004 0.30000000000000004
4500.0 0.587455439567566 0.07 0.30000000000000004 0.5
4900.0 0.6314045190811157 0.07 0.30000000000000004 0.7000000000000001
8000.0 1.030586552619934 0.07 0.30000000000000004 0.9
18100.0 2.40663366317749 0.07 0.5 0.1
6600.0 0.8307950258255005 0.07 0.5 0.30000000000000004
5700.0 0.7286819219589233 0.07 0.5 0.5
6700.0 0.8653172731399537 0.07 0.5 0.7000000000000001
10500.0 1.3530909776687623 0.07 0.5 0.9
19100.0 2.4030818939208984 0.07 0.7000000000000001 0.1
8000.0 1.0128585338592528 0.07 0.7000000000000001 0.30000000000000004
5800.0 0.7409590005874633 0.07 0.7000000000000001 0.5
6800.0 0.8729534864425659 0.07 0.7000000000000001 0.7000000000000001
10800.0 1.3907411098480225 0.07 0.7000000000000001 0.9
21100.0 2.6601486682891844 0.07 0.9 0.1
7400.0 0.9573254346847534 0.07 0.9 0.30000000000000004
6900.0 0.8853067398071289 0.07 0.9 0.5
7500.0 0.9662659645080567 0.07 0.9 0.7000000000000001
13300.0 1.7121004343032837 0.07 0.9 0.9
7000.0 0.877167010307312 0.09 0.1 0.1
3200.0 0.39817490577697756 0.09 0.1 0.30000000000000004
2100.0 0.2657524585723877 0.09 0.1 0.5
2700.0 0.3447479248046875 0.09 0.1 0.7000000000000001
3800.0 0.4851510524749756 0.09 0.1 0.9
12000.0 1.4791327953338622 0.09 0.30000000000000004 0.1
4700.0 0.6009609937667847 0.09 0.30000000000000004 0.30000000000000004
3800.0 0.48763813972473147 0.09 0.30000000000000004 0.5
4100.0 0.5299638032913208 0.09 0.30000000000000004 0.7000000000000001
6500.0 0.8335980176925659 0.09 0.30000000000000004 0.9
15400.0 1.897213053703308 0.09 0.5 0.1
5600.0 0.7076650619506836 0.09 0.5 0.30000000000000004
4600.0 0.5893559217453003 0.09 0.5 0.5
4400.0 0.5717717409133911 0.09 0.5 0.7000000000000001
8000.0 1.032082200050354 0.09 0.5 0.9
13400.0 1.6631282567977905 0.09 0.7000000000000001 0.1
6000.0 0.7739809989929199 0.09 0.7000000000000001 0.30000000000000004
5000.0 0.6384347677230835 0.09 0.7000000000000001 0.5
5200.0 0.6734694957733154 0.09 0.7000000000000001 0.7000000000000001
9200.0 1.18148832321167 0.09 0.7000000000000001 0.9
19800.0 2.4389530181884767 0.09 0.9 0.1
6800.0 0.8645286560058594 0.09 0.9 0.30000000000000004
5300.0 0.6971422433853149 0.09 0.9 0.5
5700.0 0.7404067277908325 0.09 0.9 0.7000000000000001
10900.0 1.401625657081604 0.09 0.9 0.9
6700.0 0.8235293626785278 0.11 0.1 0.1
2400.0 0.30350518226623535 0.11 0.1 0.30000000000000004
2100.0 0.26966214179992676 0.11 0.1 0.5
2200.0 0.2799672603607178 0.11 0.1 0.7000000000000001
2700.0 0.34594295024871824 0.11 0.1 0.9
8400.0 1.0439855575561523 0.11 0.30000000000000004 0.1
3700.0 0.4675200700759888 0.11 0.30000000000000004 0.30000000000000004
3200.0 0.4125957489013672 0.11 0.30000000000000004 0.5
3400.0 0.43566222190856935 0.11 0.30000000000000004 0.7000000000000001
5500.0 0.7047901630401612 0.11 0.30000000000000004 0.9
10500.0 1.3315981388092042 0.11 0.5 0.1
4700.0 0.6023809671401977 0.11 0.5 0.30000000000000004
3800.0 0.4838422775268555 0.11 0.5 0.5
4300.0 0.552120852470398 0.11 0.5 0.7000000000000001
7700.0 0.9862978458404541 0.11 0.5 0.9
11700.0 1.4546268939971925 0.11 0.7000000000000001 0.1
5000.0 0.6322080612182617 0.11 0.7000000000000001 0.30000000000000004
4300.0 0.5509825944900513 0.11 0.7000000000000001 0.5
4700.0 0.6091274499893189 0.11 0.7000000000000001 0.7000000000000001
8500.0 1.0928788661956788 0.11 0.7000000000000001 0.9
18500.0 2.270965075492859 0.11 0.9 0.1
5800.0 0.7501320362091064 0.11 0.9 0.30000000000000004
4400.0 0.5667460918426513 0.11 0.9 0.5
4500.0 0.5811226844787598 0.11 0.9 0.7000000000000001
8900.0 1.1452609300613403 0.11 0.9 0.9
6100.0 0.8339898824691773 0.13 0.1 0.1
2200.0 0.2762604236602783 0.13 0.1 0.30000000000000004
1800.0 0.23013579845428467 0.13 0.1 0.5
1800.0 0.23038487434387206 0.13 0.1 0.7000000000000001
2600.0 0.33269972801208497 0.13 0.1 0.9
7700.0 0.9480060815811158 0.13 0.30000000000000004 0.1
3800.0 0.4755197525024414 0.13 0.30000000000000004 0.30000000000000004
2600.0 0.33149802684783936 0.13 0.30000000000000004 0.5
2700.0 0.345818305015564 0.13 0.30000000000000004 0.7000000000000001
4800.0 0.6153265714645386 0.13 0.30000000000000004 0.9
10300.0 1.2720768451690674 0.13 0.5 0.1
4100.0 0.5347757816314698 0.13 0.5 0.30000000000000004
3400.0 0.43972880840301515 0.13 0.5 0.5
3500.0 0.45083327293395997 0.13 0.5 0.7000000000000001
5100.0 0.655206298828125 0.13 0.5 0.9
11800.0 1.4667126178741454 0.13 0.7000000000000001 0.1
4700.0 0.6060858726501465 0.13 0.7000000000000001 0.30000000000000004
3400.0 0.4342501640319824 0.13 0.7000000000000001 0.5
4100.0 0.5315658330917359 0.13 0.7000000000000001 0.7000000000000001
7100.0 0.9131112813949585 0.13 0.7000000000000001 0.9
11200.0 1.389155888557434 0.13 0.9 0.1
5100.0 0.6411936521530152 0.13 0.9 0.30000000000000004
4000.0 0.5180796384811401 0.13 0.9 0.5
4500.0 0.5756556034088135 0.13 0.9 0.7000000000000001
7600.0 0.9810837984085083 0.13 0.9 0.9
4600.0 0.5640867471694946 0.15 0.1 0.1
1900.0 0.23779876232147218 0.15 0.1 0.30000000000000004
1900.0 0.24053120613098145 0.15 0.1 0.5
1700.0 0.21579303741455078 0.15 0.1 0.7000000000000001
2500.0 0.3188217878341675 0.15 0.1 0.9
8100.0 0.9990598201751709 0.15 0.30000000000000004 0.1
3000.0 0.37704198360443114 0.15 0.30000000000000004 0.30000000000000004
2600.0 0.32734701633453367 0.15 0.30000000000000004 0.5
2800.0 0.35742032527923584 0.15 0.30000000000000004 0.7000000000000001
4400.0 0.5637786865234375 0.15 0.30000000000000004 0.9
8500.0 1.055244016647339 0.15 0.5 0.1
3500.0 0.4434426546096802 0.15 0.5 0.30000000000000004
2700.0 0.34559504985809325 0.15 0.5 0.5
3100.0 0.4002765417098999 0.15 0.5 0.7000000000000001
4900.0 0.6292182445526123 0.15 0.5 0.9
9800.0 1.2313106298446654 0.15 0.7000000000000001 0.1
4000.0 0.508252763748169 0.15 0.7000000000000001 0.30000000000000004
2900.0 0.3753349781036377 0.15 0.7000000000000001 0.5
3500.0 0.44790427684783934 0.15 0.7000000000000001 0.7000000000000001
5300.0 0.6813356399536132 0.15 0.7000000000000001 0.9
10900.0 1.3455046892166138 0.15 0.9 0.1
4200.0 0.5277354955673218 0.15 0.9 0.30000000000000004
3100.0 0.40350797176361086 0.15 0.9 0.5
3900.0 0.5034810543060303 0.15 0.9 0.7000000000000001
6100.0 0.7860031127929688 0.15 0.9 0.9
3800.0 0.46663694381713866 0.17 0.1 0.1
1900.0 0.2371819019317627 0.17 0.1 0.30000000000000004
1400.0 0.17777867317199708 0.17 0.1 0.5
1500.0 0.19182796478271485 0.17 0.1 0.7000000000000001
2000.0 0.25494239330291746 0.17 0.1 0.9
6300.0 0.7758348226547241 0.17 0.30000000000000004 0.1
2600.0 0.3252236843109131 0.17 0.30000000000000004 0.30000000000000004
2000.0 0.2517982482910156 0.17 0.30000000000000004 0.5
2400.0 0.3075207948684692 0.17 0.30000000000000004 0.7000000000000001
3800.0 0.4881488561630249 0.17 0.30000000000000004 0.9
8700.0 1.0704642295837403 0.17 0.5 0.1
3200.0 0.4173693895339966 0.17 0.5 0.30000000000000004
2400.0 0.3036945819854736 0.17 0.5 0.5
2500.0 0.3179716348648071 0.17 0.5 0.7000000000000001
3700.0 0.47311928272247317 0.17 0.5 0.9
7800.0 0.9753851652145386 0.17 0.7000000000000001 0.1
3200.0 0.4030888795852661 0.17 0.7000000000000001 0.30000000000000004
3300.0 0.4272929191589355 0.17 0.7000000000000001 0.5
2900.0 0.3720515727996826 0.17 0.7000000000000001 0.7000000000000001
5200.0 0.6672865629196167 0.17 0.7000000000000001 0.9
8700.0 1.0811856031417846 0.17 0.9 0.1
3700.0 0.47278261184692383 0.17 0.9 0.30000000000000004
3400.0 0.43359899520874023 0.17 0.9 0.5
3200.0 0.4141425132751465 0.17 0.9 0.7000000000000001
5200.0 0.6696152210235595 0.17 0.9 0.9
3400.0 0.41766350269317626 0.19 0.1 0.1
2000.0 0.27104122638702394 0.19 0.1 0.30000000000000004
1400.0 0.17758519649505616 0.19 0.1 0.5
1500.0 0.1917268991470337 0.19 0.1 0.7000000000000001
1800.0 0.2269843339920044 0.19 0.1 0.9
5400.0 0.6680469274520874 0.19 0.30000000000000004 0.1
2300.0 0.28707199096679686 0.19 0.30000000000000004 0.30000000000000004
2500.0 0.32055599689483644 0.19 0.30000000000000004 0.5
2000.0 0.2565619707107544 0.19 0.30000000000000004 0.7000000000000001
3900.0 0.4991289615631104 0.19 0.30000000000000004 0.9
6000.0 0.7419610738754272 0.19 0.5 0.1
2400.0 0.3006353139877319 0.19 0.5 0.30000000000000004
2500.0 0.3208888053894043 0.19 0.5 0.5
2400.0 0.3076744794845581 0.19 0.5 0.7000000000000001
4700.0 0.5993292570114136 0.19 0.5 0.9
8600.0 1.0581413507461548 0.19 0.7000000000000001 0.1
3100.0 0.39866597652435304 0.19 0.7000000000000001 0.30000000000000004
2400.0 0.30711076259613035 0.19 0.7000000000000001 0.5
3000.0 0.38606722354888917 0.19 0.7000000000000001 0.7000000000000001
4300.0 0.5532007932662963 0.19 0.7000000000000001 0.9
8000.0 0.9915069341659546 0.19 0.9 0.1
3200.0 0.4066309928894043 0.19 0.9 0.30000000000000004
2500.0 0.3199321746826172 0.19 0.9 0.5
3000.0 0.3845560073852539 0.19 0.9 0.7000000000000001
5300.0 0.6782634735107422 0.19 0.9 0.9

## The best ones were:



3500.0 0.4442156791687012 0.05 0.1 0.5
3200.0 0.409309983253479 0.05 0.1 0.7000000000000001

3000.0 0.3767707109451294 0.07 0.1 0.5
3100.0 0.39662353992462157 0.07 0.1 0.7000000000000001

3200.0 0.39817490577697756 0.09 0.1 0.30000000000000004
2100.0 0.2657524585723877 0.09 0.1 0.5
2700.0 0.3447479248046875 0.09 0.1 0.7000000000000001
3800.0 0.4851510524749756 0.09 0.1 0.9



2400.0 0.30350518226623535         0.11 0.1 0.30000000000000004
2100.0 0.26966214179992676         0.11 0.1 0.5
2200.0 0.2799672603607178         0.11 0.1 0.7000000000000001
2700.0 0.34594295024871824         0.11 0.1 0.9
3700.0 0.4675200700759888         0.11 0.30000000000000004 0.30000000000000004
3200.0 0.4125957489013672         0.11 0.30000000000000004 0.5
3400.0 0.43566222190856935         0.11 0.30000000000000004 0.7000000000000001
5500.0 0.7047901630401612         0.11 0.30000000000000004 0.9
2200.0 0.2762604236602783         0.13 0.1 0.30000000000000004
1800.0 0.23013579845428467         0.13 0.1 0.5
1800.0 0.23038487434387206         0.13 0.1 0.7000000000000001
2600.0 0.33269972801208497         0.13 0.1 0.9
2600.0 0.33149802684783936         0.13 0.30000000000000004 0.5
2700.0 0.345818305015564         0.13 0.30000000000000004 0.7000000000000001
1900.0 0.23779876232147218         0.15 0.1 0.30000000000000004
1900.0 0.24053120613098145         0.15 0.1 0.5
1700.0 0.21579303741455078         0.15 0.1 0.7000000000000001
2500.0 0.3188217878341675         0.15 0.1 0.9
3000.0 0.37704198360443114         0.15 0.30000000000000004 0.30000000000000004
2600.0 0.32734701633453367         0.15 0.30000000000000004 0.5
2800.0 0.35742032527923584         0.15 0.30000000000000004 0.7000000000000001
2700.0 0.34559504985809325         0.15 0.5 0.5
3100.0 0.4002765417098999         0.15 0.5 0.7000000000000001
2900.0 0.3753349781036377         0.15 0.7000000000000001 0.5
1900.0 0.2371819019317627         0.17 0.1 0.30000000000000004
1400.0 0.17777867317199708         0.17 0.1 0.5
1500.0 0.19182796478271485         0.17 0.1 0.7000000000000001
2000.0 0.25494239330291746         0.17 0.1 0.9
2000.0 0.2517982482910156         0.17 0.30000000000000004 0.5
2400.0 0.3075207948684692         0.17 0.30000000000000004 0.7
2900.0 0.3720515727996826         0.17 0.7000000000000001 0.700
2000.0 0.27104122638702394         0.19 0.1 0.30000000000000004
1400.0 0.17758519649505616         0.19 0.1 0.5
1500.0 0.1917268991470337         0.19 0.1 0.7000000000000001
1800.0 0.2269843339920044         0.19 0.1 0.9
2400.0 0.3006353139877319         0.19 0.5 0.30000000000000004
2500.0 0.3208888053894043         0.19 0.5 0.5
2400.0 0.3076744794845581         0.19 0.5 0.7